# Collaborative Review Task 4

## Case Study

In this assignment, you are going to be required to price a European put option using the three different techniques introduced in this module. The parameters are as follows:

- Risk-free continuously-compounded interest rate, r, of 10%
- Strike price, $ K $, of $1000

- Initial stock price, $ S_0 $, of $120

- Time to maturity, $ T $, of 2 years
- Stock volatility, $ \sigma $, of 25%

You may make all the assumptions of the standard Black-Scholes model.

1. Initialize the relevant parameters.

2. Calculate the analytical price for the put option using the closed-form solution.

3. Calculate the estimate for the put price using the characteristic function and technique described in note set 1. Use a maximum integral bound of 40 and $ N = 200 $ rectangles to approximate your integral. 

4. Observe that the payoff of a put option for the variable $ S_T = \log(\frac{S_T}{K})$ is $ p(s) = K(1-e^s)_{s<=0} $. This means that our $ v_n $ co-efficients in the COS method become:

    $ v_n = \frac{2}{b_2-b_1} \int_{b_1}^0 K(1-e^s) \cos(n \pi \frac{s-a}{b-a}) 
    = \frac{2}{b_2-b_1} K(\psi_n(b_1,0)-\theta_n(b_1,0)))$
    
Using this information, use the COS method to price an European put option using the given parameters. Calculate put prices for increasing upper limits on the COS method sum. Plot your results against the analytical put price.

5. Use the fast Fourier transform (FFT) to price an European put option with the given parameters. You will have to price your put option for a range of strikes. Note that the only difference when applying the FFT to a put, when compared to a call, is that your value for $ \alpha $ must be less than -1. Use a sequence length for your $ x_n $ of $ N = 2^10 $, and use $ \delta_u = 0.25 $. Plot the estimated put prices against the analytical put prices (across the range of strikes).

## Answer

### Import Python library

In [9]:
import numpy as np
from scipy.stats import norm, uniform
import matplotlib.pyplot as plt
import math
import random
%matplotlib inline

#### 1. Initialize the relevant parameters.

In [10]:
# Share specfic information
r = 0.1
S0 = 120
sigma = 0.25

# Put Option Specific Information
K = 1000
T = 2
k_log = np.log(K)

#### 2. Calculate the analytical price for the put option using the closed form solution

In [11]:
# 3 functions to calculate d_1,d_2, and analytical put price

def d_1_calculate(S_0,K,r,sigma,T):
    """Function for d_1 calculation"""
    a = math.log(S_0/K)
    b = (r + (sigma**2 / 2)) * T
    c = sigma * math.sqrt(T)
    result = (a + b) / c
    return result

def d_2_calculate(d_1,sigma,T):
    a = sigma * math.sqrt(T)
    result = d_1 - a
    return result

# Function for Analytical Solution
def analytic_putprice_calculate(S_0,d_1,d_2,K,r,T):
    a = -S_0 * norm.cdf(-d_1)
    b = norm.cdf(-d_2) * K * math.exp(-r * T)
    result = a + b
    return result

In [12]:
d_1 = d_1_calculate(S0,K,r,sigma,T)
d_2 = d_2_calculate(d_1,sigma,T)

analytic_putprice = analytic_putprice_calculate(S0,d_1,d_2,K,r,T)
print("The Analytical price for the Put Option using the Closed-Form Solution is: {}".format(analytic_putprice))

The Analytical price for the Put Option using the Closed-Form Solution is: 698.7307536073846


#### 3. Calculate the estimate for the put price using the characteristic function

In [13]:
# Characteristic functions
def c_M1(t):
    a = np.log(S0) + (r - sigma**2/2)*T
    b = (1/2) * (sigma**2) * T * (t**2)
    result = np.exp(1j*a - b)
    return result

def c_M2(t):
    a = 1j * sigma**2 * T
    result = np.exp(a) * c_M1(t)
    return result

In [14]:
# Choosing t_max and N
t_max = 20
N = 200

# Calculating delta and constructing t_n
delta_t = t_max/N
from_1_to_N = np.linspace(1,N,N)
t_n = (from_1_to_N-1/2)*delta_t

In [17]:
# Appriximate integral estimates

# Function calculate integral
def degree_integral(t_n,k_log,c_M_degree_diff,delta_t):
    a = np.exp(-1j*t_n*k_log)
    b = (a * c_M_degree_diff).imag
    c = b/t_n * delta_t
    result = sum(c)
    return result

first_integral = degree_integral(t_n,k_log,c_M2,delta_t)
second_integral = degree_integral(t_n,k_log,c_M1,delta_t)

TypeError: unsupported operand type(s) for *: 'complex' and 'function'

#### 4. Use the COS method to price an European put option using the given parameters. Calculate put prices for increasing upper limits on the COS method sum. Plot your results against the analytical put price. 